# Preppin' Data Challenge -
## 2023: Week 31 - HR Month - Filling in Missing IDs

## Created by: Ghafar Shah

In this challenge, we have two HR tables: one that contains the list of employees (with their name, DOB, etc.), and one that contains a monthly snapshot of which employees worked at each DC during the month.

We want to be able to join the two tables, but sometimes the employee_id is missing from one (or both!) of the tables. We do have a second ID (the guid), but sometimes that is missing as well! Fortunately, we have at least one of those IDs present for each record. 

The goal for this challenge is to fill in the missing IDs in both tables, so they can be joined later.

### Import pandas for data preparation and analysis

In [ ]:
import pandas as pd
#pd.options.mode.chained_assignment = None  # default='warn

### Read in both employee files 
- ee_dim: list of employees (with their name, DOB, etc.)
- monthly_ee: monthly snapshot of which employees worked at each DC during the month

In [ ]:
# read in list of employees with demographics info
ee_dim = pd.read_csv('ee_dim_input.csv')

# preview dataframe
ee_dim

In [ ]:
# read in monthly snapshot of employees
monthly_ee = pd.read_csv('ee_monthly_input.csv')

# preview dataframe
monthly_ee

### First, we're going to create a lookup key table with employee_id and guid fields from both files:

- Find the unique employee_id / guid combinations in each table
- Union the results together and remove any duplicates
- Filter out any rows where one of the IDs is missing

In [ ]:
# filter employee dimension file to just employee_id and guid fields
ee_dim_ids = ee_dim[["employee_id", "guid"]]

# preview dataframe / results
ee_dim_ids

In [ ]:
# filter monthly employee file to just employee_id and guid fields
monthly_ee_ids = monthly_ee[["employee_id", "guid"]]

# preview dataframe / results
monthly_ee_ids

### Place the updated dataframes with Employee_ID and GUID fields into a frame

In [ ]:
# store the updated dataframes into a variable named frames
frames = [monthly_ee_ids, ee_dim_ids]

### Stack the dataframes to get a complete list

In [ ]:
# combine the two dataframes together using concat
key_table = pd.concat(frames)
key_table

### Filter out any rows where one of the IDs is missing

In [ ]:
# remove duplicates based on all columns in key table
key_table_de_duped = key_table.drop_duplicates()
key_table_de_duped

### De-dupe the key table

In [ ]:
# Using DataFrame.dropna() method drop all rows that have NAN/none.
key_table_clean=key_table_de_duped.dropna()
key_table_clean

### Check for any NaNs in employee_id and guid fields

In [ ]:
# check total NaN values in column 'employee_id'
print(key_table_clean['employee_id'].isnull().sum())
print(key_table_clean['guid'].isnull().sum())

### Now, we will join the main table with the lookup table on employee_id (make sure to keep all records from the original table, whether or not they match)

Let's first start with the ee_dim table (employee dimensions)

### Use merge in pandas to join the Employee Dimensions (ee_dim) table to the key table

In [ ]:
# join employee dimension table to key table on the column employee_id
ee_dim_df = ee_dim.merge(key_table_clean, on='employee_id',how='left')

# preview dataframe / results
ee_dim_df

### If guid is missing from the main table, then replace it with the guid from the lookup key table

In [ ]:
# replace NaN in guid column from main table with guid value in key table
ee_dim_df.guid_x.fillna(ee_dim_df.guid_y, inplace=True)

# drop the duplicate guid column from key table
del ee_dim_df['guid_y']

# rename the guid column in main table back
ee_dim_df.rename(columns = {'guid_x':'guid'}, inplace = True)

In [ ]:
# preview employee dimension dataframe
ee_dim_df

### Next, we will join the updated dataframe above to the lookup table again but this time on guid. If the employee_id is missing from the main table, replace it with the employee_id value from the lookup table

In [ ]:
# join employee dimension table to key table on the column guid
ee_dim_df2 = ee_dim_df.merge(key_table_clean, on='guid',how='left')

# preview dataframe / results
ee_dim_df2

In [ ]:
# replace NaN in employee_id column from main table with employee_id value in key table
ee_dim_df2.employee_id_x.fillna(ee_dim_df2.employee_id_y, inplace=True)

# drop the duplicate employee_id column from key table
del ee_dim_df2['employee_id_y']

# rename the employee_id_x column in main table back to employee_id
ee_dim_df2.rename(columns = {'employee_id_x':'employee_id'}, inplace = True)

In [ ]:
# preview updated dataframe / results
ee_dim_df2

### Finally, we will check for any duplicate rows and if there are any duplicate GUIDs / Employee IDs

In [ ]:
# checks for duplicate rows
duplicate_ee_dim_df2 = ee_dim_df2[ee_dim_df2.duplicated()]
 
print("Duplicate Rows :")
 
# Print the resultant Dataframe
duplicate_ee_dim_df2

In [ ]:
# checks for any duplicate employee_ids
ee_id_duplicate_check = ee_dim_df2.duplicated(subset=['employee_id']).any()

# checks for any duplicate guids
guid_duplicate_check = ee_dim_df2.duplicated(subset=['guid']).any()

# returns boolean true / false - we're expecting false which means there are no duplicates in either columns
ee_id_duplicate_check
guid_duplicate_check

### Repeat the same steps above but this time for Monthly Employee Roster (monthly_ee)
- Note: we do not need to do any duplicate checks here as an employee can appear more than once in a monthly snapshot report

In [ ]:
# join monthly employee table to key table on column employee_id
monthly_ee_df = monthly_ee.merge(key_table_clean, on='employee_id',how='left')

# preview dataframe
monthly_ee_df

In [ ]:
# fill in NaN with guid value from key table
monthly_ee_df.guid_x.fillna(monthly_ee_df.guid_y, inplace=True)

# drop the duplicate guid column from key table
del monthly_ee_df['guid_y']

# rename original guid_x back to guid on main table
monthly_ee_df.rename(columns = {'guid_x':'guid'}, inplace = True)

# preview dataframe / results
monthly_ee_df

In [ ]:
# join updated monthly employee table to key table on the column guid
monthly_ee_df2 = monthly_ee_df.merge(key_table_clean, on='guid',how='left')
monthly_ee_df2

In [ ]:
# fill in NaN with employee_id value from key table
monthly_ee_df2.employee_id_x.fillna(monthly_ee_df2.employee_id_y, inplace=True)

# drop the duplicate employee_id column from key table
del monthly_ee_df2['employee_id_y']

# rename original employee_id_x back to employee_id in the main table
monthly_ee_df2.rename(columns = {'employee_id_x':'employee_id'}, inplace = True)

# preview updated dataframe / results
monthly_ee_df2

### Exporting the DataFrames into a CSV file
Uncomment code to export the data as CSV

In [ ]:
# monthly_ee_df2.to_csv('Final_Monthly_EE_Roster.csv')
# ee_dim_df2.to_csv('Final_Employee_Dimensions_Roster.csv')

-----------------------------------------------------------------


# Exploring seaborn for statistical visualization


-----------------------------------------------------------------

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# join updated monthly employee table to updated dimensions on the column employee_id
monthly_ee_df3 = monthly_ee_df2.merge(ee_dim_df2, on='employee_id',how='left')
monthly_ee_df3

In [ ]:
# keep specific colums for data viz
monthly_ee_df4 = monthly_ee_df3[["employee_id", "hire_date_y", "gender"]]
monthly_ee_df4

In [ ]:
# keep the year after last backslash
monthly_ee_df4 = monthly_ee_df4.assign(Years=monthly_ee_df4.hire_date_y.str[-4:])

monthly_ee_df4['Years'] = pd.to_numeric(monthly_ee_df4['Years'], errors='coerce')

In [ ]:
monthly_ee_df4

In [ ]:
# check datatypes
monthly_ee_df4.dtypes

In [ ]:
# delete hire_date_y column
del monthly_ee_df4['hire_date_y']

# preview dataframe / results
monthly_ee_df4

In [ ]:
# removes duplicates based on all columns
dataviz_monthly_ee = monthly_ee_df4.drop_duplicates(subset=['employee_id', 'gender', 'Years'])

In [ ]:
# preview dataframe
dataviz_monthly_ee

### Add labels to bar chart
Note: The data labels source code came from # sources from https://www.statology.org/seaborn-barplot-show-values/

In [ ]:
# source: https://www.statology.org/seaborn-barplot-show-values/

def show_values(axs, orient="v", space=.01):
    def _single(ax):
        if orient == "v":
            for p in ax.patches:
                _x = p.get_x() + p.get_width() / 2
                _y = p.get_y() + p.get_height() + (p.get_height()*0.01)
                value = '{:.0f}'.format(p.get_height())
                ax.text(_x, _y, value, ha="center") 
        elif orient == "h":
            for p in ax.patches:
                _x = p.get_x() + p.get_width() + float(space)
                _y = p.get_y() + p.get_height() - (p.get_height()*0.5)
                value = '{:.1f}'.format(p.get_width())
                ax.text(_x, _y, value, ha="left")

    if isinstance(axs, np.ndarray):
        for idx, ax in np.ndenumerate(axs):
            _single(ax)
    else:
        _single(axs)

### Barplot shows number of employees hired by year

In [ ]:
#create vertical barplot
p = sns.countplot(x=dataviz_monthly_ee["Years"], color='blue')
p.set(xlabel='Hire Date', ylabel='Number of Employees')
p.set_title('Number of Employees Hired By Year')

#show values on barplot
show_values(p)

# export / save barplot as image
p.figure.savefig('bar_plot.png', transparent = True)

### Stacked barplot shows employees hired by gender by year

In [ ]:
p = sns.histplot(
    data=dataviz_monthly_ee,
    x="Years", hue="gender",
    multiple="fill", stat="count",
    discrete=True, shrink=.9
).set(title='Visualizing Gender Diversity in Yearly Hires')

plt.xlabel("Hire Date")
plt.ylabel("Proportion")

# Export the plot to an image file
plt.savefig('histogram_plot.png', transparent =True)

plt.show()

### Export dataviz_monthly_ee file to CSV
Uncomment to export data to CSV

In [ ]:
# dataviz_monthly_ee.to_csv('dataviz_monthly_ee_file.csv')